In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense, Input
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.
/usr/local/lib/python3.5/dist-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 6021 on context None
Preallocating 3633/4037 Mb (0.900000) on cuda1
Mapped name None to device cuda1: GeForce GTX 970 (0000:02:00.0)


In [2]:
input_tensor = Input(shape=(3, 256, 256))
base_model = InceptionV3(weights='imagenet', include_top=False, 
                         input_tensor=input_tensor)

In [3]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [4]:
model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
for layer in base_model.layers:
    layer.trainable = False

In [6]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
path = 'data/'

In [8]:
gen = ImageDataGenerator()
batches = gen.flow_from_directory(path + 'train')
valid_batches = gen.flow_from_directory(path + 'valid')

Found 2067 images belonging to 2 classes.
Found 228 images belonging to 2 classes.


In [9]:
batch_size = 32

In [14]:
model.fit_generator(batches, int(batches.samples/batch_size), 
                    epochs=15, validation_data=valid_batches, 
                    validation_steps=int(valid_batches.samples/batch_size))

Epoch 1/15
64/64 [==============================] - 30s - loss: 0.1881 - acc: 0.9261 - val_loss: 0.2374 - val_acc: 0.8980
Epoch 2/15
64/64 [==============================] - 27s - loss: 0.1718 - acc: 0.9370 - val_loss: 0.2985 - val_acc: 0.8622
Epoch 3/15
64/64 [==============================] - 26s - loss: 0.1703 - acc: 0.9302 - val_loss: 0.3122 - val_acc: 0.8878
Epoch 4/15
64/64 [==============================] - 27s - loss: 0.1561 - acc: 0.9337 - val_loss: 0.2926 - val_acc: 0.8929
Epoch 5/15
64/64 [==============================] - 26s - loss: 0.1870 - acc: 0.9311 - val_loss: 0.2603 - val_acc: 0.9286
Epoch 6/15
64/64 [==============================] - 26s - loss: 0.1605 - acc: 0.9425 - val_loss: 0.2026 - val_acc: 0.9031
Epoch 7/15
64/64 [==============================] - 26s - loss: 0.1526 - acc: 0.9390 - val_loss: 0.2972 - val_acc: 0.8827
Epoch 8/15
64/64 [==============================] - 26s - loss: 0.1379 - acc: 0.9406 - val_loss: 0.3505 - val_acc: 0.8827
Epoch 9/15
64/64 [======

In [15]:
for layer in model.layers[:249]:
    layer.trainable = False

for layer in model.layers[249:]:
    layer.trainable = True

In [16]:
from keras.optimizers import SGD

In [17]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit_generator(batches, int(batches.samples/batch_size), 
                    epochs=30, validation_data=valid_batches, 
                    validation_steps=int(valid_batches.samples/batch_size))

Epoch 1/30
64/64 [==============================] - 30s - loss: 0.0593 - acc: 0.9824 - val_loss: 0.2046 - val_acc: 0.9184
Epoch 2/30
64/64 [==============================] - 27s - loss: 0.0586 - acc: 0.9785 - val_loss: 0.1878 - val_acc: 0.9235
Epoch 3/30
64/64 [==============================] - 27s - loss: 0.0502 - acc: 0.9839 - val_loss: 0.2385 - val_acc: 0.9082
Epoch 4/30
64/64 [==============================] - 27s - loss: 0.0474 - acc: 0.9868 - val_loss: 0.2347 - val_acc: 0.9082
Epoch 5/30
64/64 [==============================] - 27s - loss: 0.0428 - acc: 0.9893 - val_loss: 0.2450 - val_acc: 0.9031
Epoch 6/30
64/64 [==============================] - 27s - loss: 0.0562 - acc: 0.9810 - val_loss: 0.1798 - val_acc: 0.9235
Epoch 7/30
64/64 [==============================] - 27s - loss: 0.0408 - acc: 0.9897 - val_loss: 0.2470 - val_acc: 0.9184
Epoch 8/30
64/64 [==============================] - 27s - loss: 0.0366 - acc: 0.9912 - val_loss: 0.2099 - val_acc: 0.9286
Epoch 9/30
64/64 [======

In [ ]:
model.save_weights(path + 'inception.h5')